In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
# kagglehub.login()
# //read from kaggle.json
from kagglehub.config import set_kaggle_credentials

import json
with open('kaggle.json', 'r') as f:
    kaggle_creds = json.load(f)
    
# kagglehub.login(username=kaggle_creds['username'], key=kaggle_creds['key'])
set_kaggle_credentials(username=kaggle_creds['username'], api_key=kaggle_creds['key'])


Kaggle credentials set.


/Users/mahta/Projects/Time-Series-Library/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

# drw_crypto_market_prediction_path = kagglehub.competition_download('drw-crypto-market-prediction')

# print('Data source import complete.')
drw_crypto_market_prediction_path = '/Users/mahta/Projects/Time-Series-Library/data/drw-crypto-market-prediction'


In [3]:
drw_crypto_market_prediction_path

'/Users/mahta/Projects/Time-Series-Library/data/drw-crypto-market-prediction'

In [4]:
"""Competition: DRW Crypto Market Prediction | Date: Week 3 | Purpose: Add Back X174"""

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from scipy.stats import pearsonr

def reduce_mem_usage(dataframe, dataset):
   print('Reducing memory usage for:', dataset)
   initial_mem_usage = dataframe.memory_usage().sum() / 1024**2

   for col in dataframe.columns:
       col_type = dataframe[col].dtype
       c_min = dataframe[col].min()
       c_max = dataframe[col].max()

       if str(col_type)[:3] == 'int':
           if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
               dataframe[col] = dataframe[col].astype(np.int8)
           elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
               dataframe[col] = dataframe[col].astype(np.int16)
           elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
               dataframe[col] = dataframe[col].astype(np.int32)
           elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
               dataframe[col] = dataframe[col].astype(np.int64)
       else:
           if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
               dataframe[col] = dataframe[col].astype(np.float16)
           elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
               dataframe[col] = dataframe[col].astype(np.float32)
           else:
               dataframe[col] = dataframe[col].astype(np.float64)

   final_mem_usage = dataframe.memory_usage().sum() / 1024**2
   print('--- Memory usage before: {:.2f} MB'.format(initial_mem_usage))
   print('--- Memory usage after: {:.2f} MB'.format(final_mem_usage))
   print('--- Decreased memory usage by {:.1f}%\n'.format(100 * (initial_mem_usage - final_mem_usage) / initial_mem_usage))

   return dataframe

def create_time_weights(n_samples, decay_factor=0.95):
   """Create exponentially decaying weights based on sample position."""
   positions = np.arange(n_samples)
   normalized_positions = positions / (n_samples - 1)
   weights = decay_factor ** (1 - normalized_positions)
   weights = weights * n_samples / weights.sum()
   return weights

# Load data
train = pd.read_parquet(drw_crypto_market_prediction_path + '/train.parquet')
test = pd.read_parquet(drw_crypto_market_prediction_path + '/test.parquet')
sample_submission = pd.read_csv(drw_crypto_market_prediction_path + '/sample_submission.csv')

# Add back X174 (lowest AV importance of removed features)
selected_features = [
   'X863', 'X856', 'X344', 'X598', 'X862', 'X385', 'X852', 'X603', 'X860',
   'X415', 'X345', 'X137', 'X855', 'X178', 'X532', 'X168', 'X174',
   'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume'
]

train = train[selected_features + ["label"]]
test = test[selected_features]

train = reduce_mem_usage(train, "train")
test = reduce_mem_usage(test, "test")

print("Train=", train.shape)
print("Test=", test.shape)

FEATURES = [c for c in train.columns if c not in ["label"]]
print(f"There are {len(FEATURES)} FEATURES (added back X174)")

# Cross-validation
FOLDS = 5
kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

# Create bins for stratification
train['label_float32'] = train['label'].astype(np.float32)
train['label_bins'] = pd.qcut(train['label_float32'], q=10, labels=False, duplicates='drop')
train = train.drop('label_float32', axis=1)

# XGBoost parameters
xgb_params = {
   "tree_method": "hist",
   "device": "cuda",
   "colsample_bylevel": 0.4778015829774066,
   "colsample_bynode": 0.362764358742407,
   "colsample_bytree": 0.7107423488010493,
   "gamma": 1.7094857725240398,
   "learning_rate": 0.02213323588455387,
   "max_depth": 20,
   "max_leaves": 12,
   "min_child_weight": 16,
   "n_estimators": 1667,
   "n_jobs": -1,
   "random_state": 42,
   "reg_alpha": 39.352415706891264,
   "reg_lambda": 75.44843704068275,
   "subsample": 0.06566669853471274,
   "verbosity": 0,
   "objective": "reg:squarederror"
}

# Define model configurations
model_configs = [
   {"name": "Model 1 (100% Full Data)", "percent": 1.00},
   {"name": "Model 2 (90% Recent)", "percent": 0.90},
   {"name": "Model 3 (80% Recent)", "percent": 0.80},
   {"name": "Model 4 (70% Recent)", "percent": 0.70},
   {"name": "Model 5 (60% Recent)", "percent": 0.60},
   {"name": "Model 6 (50% Recent)", "percent": 0.50},
   {"name": "Model 7 (40% Recent)", "percent": 0.40}
]

# Initialize predictions
n_models = len(model_configs)
oof_preds_all = [np.zeros(len(train)) for _ in range(n_models)]
test_preds_all = [np.zeros(len(test)) for _ in range(n_models)]

# Generate sample weights for full data
sample_weights_full = create_time_weights(len(train), decay_factor=0.95)
print(f"\nFull data sample weights range: [{sample_weights_full.min():.4f}, {sample_weights_full.max():.4f}]")

# Calculate cutoffs
cutoffs = []
for config in model_configs:
   if config["percent"] == 1.00:
       cutoffs.append(0)
   else:
       cutoff_idx = int(len(train) * (1 - config["percent"]))
       cutoffs.append(cutoff_idx)
       print(f"{config['name']} - Using most recent {len(train) - cutoff_idx} samples")

# Cross-validation loop
for fold_num, (train_idx, valid_idx) in enumerate(kf.split(train, train['label_bins'])):
   print("\n" + "#" * 50)
   print(f"### Fold {fold_num + 1}")
   print("#" * 50)

   X_valid = train.iloc[valid_idx][FEATURES]
   y_valid = train.iloc[valid_idx]["label"]
   X_test = test[FEATURES]

   # Train each model
   for model_idx, (config, cutoff) in enumerate(zip(model_configs, cutoffs)):
       print(f"\n--- {config['name']} ---")

       if config["percent"] == 1.00:
           X_train = train.iloc[train_idx][FEATURES]
           y_train = train.iloc[train_idx]["label"]
           train_weights = sample_weights_full[train_idx]
       else:
           train_idx_recent = train_idx[train_idx >= cutoff]
           train_idx_recent_adjusted = train_idx_recent - cutoff
           train_recent = train.iloc[cutoff:].reset_index(drop=True)

           X_train = train_recent.iloc[train_idx_recent_adjusted][FEATURES]
           y_train = train_recent.iloc[train_idx_recent_adjusted]["label"]

           sample_weights_recent = create_time_weights(len(train_recent), decay_factor=0.95)
           train_weights = sample_weights_recent[train_idx_recent_adjusted]

       # Train model
       model = xgb.XGBRegressor(**xgb_params, early_stopping_rounds=25)
       model.fit(
           X_train, y_train,
           sample_weight=train_weights,
           eval_set=[(X_valid, y_valid)],
           verbose=200
       )

       # Make predictions
       if config["percent"] == 1.00:
           oof_preds_all[model_idx][valid_idx] = model.predict(X_valid)
       else:
           valid_idx_in_range = valid_idx[valid_idx >= cutoff]
           if len(valid_idx_in_range) > 0:
               X_valid_subset = train.iloc[valid_idx_in_range][FEATURES]
               oof_preds_all[model_idx][valid_idx_in_range] = model.predict(X_valid_subset)

           valid_idx_out_range = valid_idx[valid_idx < cutoff]
           if len(valid_idx_out_range) > 0:
               oof_preds_all[model_idx][valid_idx_out_range] = oof_preds_all[0][valid_idx_out_range]

       test_preds_all[model_idx] += model.predict(X_test)

# Average test predictions across folds
for i in range(n_models):
   test_preds_all[i] /= FOLDS

# Calculate individual model scores
pearson_scores = []
for i, config in enumerate(model_configs):
   score = pearsonr(train["label"], oof_preds_all[i])[0]
   pearson_scores.append(score)

print("\n" + "=" * 50)
print("INDIVIDUAL MODEL PERFORMANCE (+X174)")
print("=" * 50)
for config, score in zip(model_configs, pearson_scores):
   print(f"{config['name']} Pearson Correlation: {score:.4f}")

# Create ensemble predictions
ensemble_oof_preds = np.mean(oof_preds_all, axis=0)
ensemble_test_preds = np.mean(test_preds_all, axis=0)

ensemble_pearson_score = pearsonr(train["label"], ensemble_oof_preds)[0]
print(f"\nEnsemble (Equal Weight) Pearson Correlation: {ensemble_pearson_score:.4f}")

# Weighted ensemble
total_score = sum(pearson_scores)
weights = [score / total_score for score in pearson_scores]

weighted_ensemble_oof = np.zeros(len(train))
weighted_ensemble_test = np.zeros(len(test))

for i in range(n_models):
   weighted_ensemble_oof += weights[i] * oof_preds_all[i]
   weighted_ensemble_test += weights[i] * test_preds_all[i]

weighted_ensemble_score = pearsonr(train["label"], weighted_ensemble_oof)[0]
print(f"Weighted Ensemble Pearson Correlation: {weighted_ensemble_score:.4f}")

# Use the better ensemble
if weighted_ensemble_score > ensemble_pearson_score:
   final_test_preds = weighted_ensemble_test
   print("\nUsing weighted ensemble for final predictions")
else:
   final_test_preds = ensemble_test_preds
   print("\nUsing simple average ensemble for final predictions")


Reducing memory usage for: train
--- Memory usage before: 96.29 MB
--- Memory usage after: 27.08 MB
--- Decreased memory usage by 71.9%

Reducing memory usage for: test
--- Memory usage before: 90.33 MB
--- Memory usage after: 22.58 MB
--- Decreased memory usage by 75.0%

Train= (525887, 23)
Test= (538150, 22)
There are 22 FEATURES (added back X174)

Full data sample weights range: [0.9746, 1.0259]
Model 2 (90% Recent) - Using most recent 473299 samples
Model 3 (80% Recent) - Using most recent 420710 samples
Model 4 (70% Recent) - Using most recent 368121 samples
Model 5 (60% Recent) - Using most recent 315533 samples
Model 6 (50% Recent) - Using most recent 262944 samples
Model 7 (40% Recent) - Using most recent 210355 samples

##################################################
### Fold 1
##################################################

--- Model 1 (100% Full Data) ---
[0]	validation_0-rmse:1.01065
[200]	validation_0-rmse:0.98346
[400]	validation_0-rmse:0.96708
[600]	validation_0-r

In [5]:

# Save predictions
sample_sub = pd.read_csv('/Users/mahta/Projects/Time-Series-Library/data/drw-crypto-market-prediction/sample_submission.csv')
submission = pd.DataFrame({
   sample_sub.columns[0]: sample_sub.iloc[:, 0],
   'prediction': final_test_preds
})
submission.to_csv("submission_with_X174.csv", index=False)
print("\nSubmission saved to submission_with_X174.csv!")

# Gap-first validation
cv_scores = [np.sqrt(np.mean((train["label"] - oof_preds_all[i])**2)) for i in range(n_models)]
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

print(f"\n=== GAP-FIRST VALIDATION ===")
print(f"Ensemble Pearson: {weighted_ensemble_score:.4f}")
print(f"Model RMSE Std: {cv_std:.6f}")
print(f"Features: {len(FEATURES)} (added back X174)")

if cv_std < 0.01 and weighted_ensemble_score > 0.4:
   print("✅ SUBMIT: High correlation and stable models")
else:
   print("⚠️  CAUTION: Review before submission")


Submission saved to submission_with_X174.csv!

=== GAP-FIRST VALIDATION ===
Ensemble Pearson: 0.5100
Model RMSE Std: 0.004741
Features: 22 (added back X174)
✅ SUBMIT: High correlation and stable models


## Timeseries Time!

In [ ]:
"""
Time-Series Transformer Benchmark for Crypto Prediction
Let's prepare the data and benchmark multiple transformer models from the library
"""

import os
import sys
import argparse
from types import SimpleNamespace
import warnings
import time
import gc
warnings.filterwarnings('ignore')

# Import the Time-Series-Library modules
sys.path.append('/Users/mahta/Projects/Time-Series-Library')

# Create a custom dataset class for our crypto data
class CryptoDataset(Dataset):
    def __init__(self, data, seq_len=96, label_len=48, pred_len=24):
        self.data = data
        self.seq_len = seq_len
        self.label_len = label_len  
        self.pred_len = pred_len
        
        # Prepare features (everything except label)
        self.features = [col for col in data.columns if col != 'label']
        self.data_x = data[self.features].values
        self.data_y = data['label'].values.reshape(-1, 1)
        
        # Normalize the data
        self.data_x = (self.data_x - self.data_x.mean(axis=0)) / (self.data_x.std(axis=0) + 1e-8)
        self.data_y = (self.data_y - self.data_y.mean()) / (self.data_y.std() + 1e-8)
        
    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1
    
    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len
        
        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        
        # Create dummy time marks (zeros for now)
        seq_x_mark = np.zeros((self.seq_len, 1))
        seq_y_mark = np.zeros((self.label_len + self.pred_len, 1))
        
        return seq_x, seq_y, seq_x_mark, seq_y_mark



# List of transformer models to benchmark
transformer_models = [
    'Transformer', 'Informer', 'Autoformer', 'FEDformer', 
    'TimesNet', 'PatchTST', 'iTransformer', 'Mamba',
    'TimeMixer', 'TSMixer', 'Crossformer', 'ETSformer'
]


⚠️ Import error: No module named 'sktime.datasets'
Some functionality may be limited, but we'll continue with available components.
✅ Time-Series Library setup complete!
Available models for benchmarking...
Models to benchmark: ['Transformer', 'Informer', 'Autoformer', 'FEDformer', 'TimesNet', 'PatchTST', 'iTransformer', 'Mamba', 'TimeMixer', 'TSMixer', 'Crossformer', 'ETSformer']


In [ ]:
# load the train_ts and all from the preprocessed daa 


# chackpoint name:


# load the model

# Inference

# load the train_ts and all from the preprocessed data 
import os
import sys
import torch
import numpy as np
import pandas as pd
import argparse
from datetime import datetime

# Add the Time-Series-Library to path
sys.path.append('/Users/mahta/Projects/Time-Series-Library')

# Import the crypto experiment class
from crypto.crypto_exp import CryptoExp_Long_Term_Forecast
from utils.print_args import print_args
from utils.metrics import metric

# checkpoint name:
checkpoint_name = "iTransformer_job46505243"
checkpoint_path = f"/Users/mahta/Projects/Time-Series-Library/checkpoints/{checkpoint_name}"

print(f"Loading checkpoint from: {checkpoint_path}")

# Create test arguments matching the training configuration
test_args = argparse.Namespace()

# Basic config
test_args.task_name = 'long_term_forecast'
test_args.is_training = 0  # Testing mode
test_args.model_id = 'crypto_test'
test_args.model = 'iTransformer'
test_args.data = 'custom'

# Data loader
test_args.root_path = './crypto/dataset/'
test_args.data_path = 'crypto.csv'
test_args.features = 'M'
test_args.target = 'label'
test_args.freq = 'h'
test_args.checkpoints = './checkpoints/'

# Forecasting task (from checkpoint name: sl96_ll48_pl24)
test_args.seq_len = 96
test_args.label_len = 48
test_args.pred_len = 24

# Model define (from checkpoint name: dm256_nh8_el2_dl1_df256)
test_args.enc_in = 24
test_args.dec_in = 24
test_args.c_out = 1
test_args.d_model = 256
test_args.n_heads = 8
test_args.e_layers = 2
test_args.d_layers = 1
test_args.d_ff = 256  # Note: capped at 256 for iTransformer
test_args.moving_avg = 25
test_args.factor = 1
test_args.distil = True
test_args.dropout = 0.1
test_args.embed = 'timeF'
test_args.activation = 'gelu'

# Optimization
test_args.num_workers = 10
test_args.itr = 1
test_args.batch_size = 32
test_args.learning_rate = 0.0001
test_args.des = 'test'
test_args.loss = 'MSE'
test_args.lradj = 'type1'
test_args.use_amp = False
test_args.inverse = True
test_args.use_dtw = False

# GPU
test_args.use_gpu = True
test_args.gpu = 0
test_args.use_multi_gpu = False
test_args.devices = '0,1,2,3'

# Check GPU availability
if torch.cuda.is_available():
    test_args.device = torch.device('cuda:{}'.format(test_args.gpu))
    print('GPU available: {}'.format(torch.cuda.get_device_name(test_args.gpu)))
else:
    print('Using CPU for inference')
    test_args.device = torch.device('cpu')

print('Model configuration:')
print_args(test_args)

# load the model
print(f'\n>>>>>>>Loading model from checkpoint: {checkpoint_name}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')

# Create experiment
exp = CryptoExp_Long_Term_Forecast(test_args)

# Test the model (this loads the checkpoint and runs inference)
preds, trues = exp.test(checkpoint_name, test=1)  # test=1 means load checkpoint

print(f'Inference completed!')
print(f'Predictions shape: {preds.shape}')
print(f'Ground truth shape: {trues.shape}')

# Calculate metrics
mae, mse, rmse, mape, mspe = metric(preds, trues)

print(f'\nTest Results:')
print(f'  MSE: {mse:.6f}')
print(f'  MAE: {mae:.6f}')
print(f'  RMSE: {rmse:.6f}')
print(f'  MAPE: {mape:.6f}')
print(f'  MSPE: {mspe:.6f}')

# Inference
print(f'\nGenerating predictions for test data...')

# The predictions are already generated above in the exp.test() call
# preds contains the model predictions for the test set

# save the predictions
print(f'\nSaving predictions...')

# Create results directory
results_dir = f'./itransformer_results/{checkpoint_name}/'
os.makedirs(results_dir, exist_ok=True)

# Save predictions and ground truth as numpy arrays
np.save(os.path.join(results_dir, 'predictions.npy'), preds)
np.save(os.path.join(results_dir, 'ground_truth.npy'), trues)

# Create summary DataFrame
summary_df = pd.DataFrame({
    'metric': ['MSE', 'MAE', 'RMSE', 'MAPE', 'MSPE'],
    'value': [mse, mae, rmse, mape, mspe]
})

summary_df.to_csv(os.path.join(results_dir, 'metrics.csv'), index=False)

# Save predictions in the same format as the original XGBoost submission
# We need to reshape predictions to match the test data format
# preds shape is (n_samples, pred_len, n_features) - we want the last feature (target)
test_predictions = preds[:, :, -1]  # Take the last feature (target variable)

# Reshape to match the test data length
# The test data has 538150 samples, but our predictions might be different
# We need to handle this properly
print(f'Test predictions shape: {test_predictions.shape}')

# If we have fewer predictions than test samples, we need to handle this
# For now, let's save what we have and create a submission file
if test_predictions.shape[0] <= len(test):
    # Create submission DataFrame
    submission = pd.DataFrame({
        'id': range(len(test_predictions)),
        'prediction': test_predictions.flatten()  # Flatten to 1D array
    })
    
    submission_file = os.path.join(results_dir, 'itransformer_submission.csv')
    submission.to_csv(submission_file, index=False)
    print(f'Submission saved to: {submission_file}')
    
    # Also save in the same format as the original XGBoost
    sample_sub = pd.read_csv('/Users/mahta/Projects/Time-Series-Library/data/drw-crypto-market-prediction/sample_submission.csv')
    submission_formatted = pd.DataFrame({
        sample_sub.columns[0]: sample_sub.iloc[:len(test_predictions), 0],
        'prediction': test_predictions.flatten()
    })
    submission_formatted.to_csv(os.path.join(results_dir, 'itransformer_submission_formatted.csv'), index=False)
    print(f'Formatted submission saved to: {os.path.join(results_dir, "itransformer_submission_formatted.csv")}')
else:
    print(f'Warning: More predictions ({test_predictions.shape[0]}) than test samples ({len(test)})')

print(f'\nAll results saved to: {results_dir}')
print('iTransformer inference completed successfully!')

In [ ]:
# Save benchmark results and create visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Save detailed results to CSV
results_df.to_csv('transformer_benchmark_results.csv', index=False)
print("💾 Benchmark results saved to 'transformer_benchmark_results.csv'")

# Create summary visualization if we have successful results
if len(successful_results) > 0:
    plt.figure(figsize=(15, 10))
    
    # 1. Correlation comparison
    plt.subplot(2, 3, 1)
    models = successful_results['model'].tolist()
    correlations = successful_results['correlation'].tolist()
    
    bars = plt.bar(range(len(models)), correlations, alpha=0.7, color='skyblue')
    plt.xlabel('Models')
    plt.ylabel('Correlation')
    plt.title('Model Correlation Comparison')
    plt.xticks(range(len(models)), models, rotation=45)
    
    # Add XGBoost baseline line
    plt.axhline(y=weighted_ensemble_score, color='red', linestyle='--', 
                label=f'XGBoost Baseline ({weighted_ensemble_score:.4f})')
    plt.legend()
    
    # Highlight best model
    best_idx = 0
    bars[best_idx].set_color('gold')
    bars[best_idx].set_alpha(1.0)
    
    # 2. Training time comparison  
    plt.subplot(2, 3, 2)
    training_times = successful_results['training_time'].tolist()
    plt.bar(range(len(models)), training_times, alpha=0.7, color='lightcoral')
    plt.xlabel('Models')
    plt.ylabel('Training Time (seconds)')
    plt.title('Training Time Comparison')
    plt.xticks(range(len(models)), models, rotation=45)
    
    # 3. RMSE comparison
    plt.subplot(2, 3, 3)
    rmse_values = successful_results['rmse'].tolist()
    plt.bar(range(len(models)), rmse_values, alpha=0.7, color='lightgreen')
    plt.xlabel('Models')
    plt.ylabel('RMSE')
    plt.title('RMSE Comparison (Lower is Better)')
    plt.xticks(range(len(models)), models, rotation=45)
    
    # 4. Model parameters vs Performance
    plt.subplot(2, 3, 4)
    params = [int(str(p).replace(',', '')) if isinstance(p, str) else p for p in successful_results['total_params']]
    plt.scatter(params, correlations, alpha=0.7, s=100, color='purple')
    plt.xlabel('Total Parameters')
    plt.ylabel('Correlation')
    plt.title('Model Size vs Performance')
    
    # Add model labels
    for i, model in enumerate(models):
        plt.annotate(model, (params[i], correlations[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # 5. Efficiency Analysis (Correlation per minute)
    plt.subplot(2, 3, 5)
    efficiency_scores = [corr / (time/60) for corr, time in zip(correlations, training_times)]
    bars = plt.bar(range(len(models)), efficiency_scores, alpha=0.7, color='orange')
    plt.xlabel('Models')
    plt.ylabel('Correlation per Minute')
    plt.title('Training Efficiency')
    plt.xticks(range(len(models)), models, rotation=45)
    
    # Highlight most efficient
    most_efficient_idx = efficiency_scores.index(max(efficiency_scores))
    bars[most_efficient_idx].set_color('darkorange')
    bars[most_efficient_idx].set_alpha(1.0)
    
    # 6. Summary radar chart for top 3 models
    plt.subplot(2, 3, 6)
    top_3_models = successful_results.head(3)
    
    if len(top_3_models) >= 2:
        # Normalize metrics for radar chart
        metrics = ['correlation', 'rmse']
        normalized_data = []
        
        for _, model_row in top_3_models.iterrows():
            # Normalize correlation (higher is better)
            norm_corr = (model_row['correlation'] - successful_results['correlation'].min()) / (successful_results['correlation'].max() - successful_results['correlation'].min())
            # Normalize RMSE (lower is better, so invert)
            norm_rmse = 1 - (model_row['rmse'] - successful_results['rmse'].min()) / (successful_results['rmse'].max() - successful_results['rmse'].min())
            normalized_data.append([norm_corr, norm_rmse])
        
        x = range(len(metrics))
        for i, (_, model_row) in enumerate(top_3_models.iterrows()):
            plt.plot(x, normalized_data[i], 'o-', label=model_row['model'], alpha=0.7)
        
        plt.xlabel('Metrics')
        plt.ylabel('Normalized Score')
        plt.title('Top 3 Models Comparison')
        plt.xticks(x, ['Correlation↑', 'RMSE↓'])
        plt.legend()
        plt.ylim(0, 1)
    else:
        plt.text(0.5, 0.5, 'Need at least 2 successful models\\nfor comparison', 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Model Comparison')
    
    plt.tight_layout()
    plt.savefig('transformer_benchmark_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("📈 Visualization saved as 'transformer_benchmark_visualization.png'")

# Create final summary report
print("\\n" + "="*80)
print("📋 FINAL BENCHMARK SUMMARY")
print("="*80)

successful_count = len(successful_results)
total_count = len(results_df)

print(f"🎯 Models tested: {total_count}")
print(f"✅ Successful: {successful_count}")
print(f"❌ Failed: {total_count - successful_count}")

if successful_count > 0:
    print(f"\\n🏆 Best performing model: {successful_results.iloc[0]['model']}")
    print(f"📊 Highest correlation: {successful_results.iloc[0]['correlation']:.6f}")
    
    # Find most efficient model
    if len(successful_results) > 1:
        efficiency_scores = successful_results['correlation'] / (successful_results['training_time'] / 60)
        most_efficient_idx = efficiency_scores.idxmax()
        most_efficient_model = successful_results.loc[most_efficient_idx, 'model']
        print(f"⚡ Most efficient model: {most_efficient_model}")
        
    print(f"\\n💡 Key insights:")
    print(f"   - Average correlation: {successful_results['correlation'].mean():.4f}")
    print(f"   - Average training time: {successful_results['training_time'].mean():.1f}s")
    print(f"   - XGBoost baseline: {weighted_ensemble_score:.4f}")
    
    # Recommendation
    best_corr = successful_results.iloc[0]['correlation']
    if best_corr > weighted_ensemble_score:
        print(f"\\n🚀 RECOMMENDATION: Use {successful_results.iloc[0]['model']} for improved performance!")
    else:
        print(f"\\n💭 RECOMMENDATION: Consider ensemble of transformers + XGBoost for best results")

print("\\n🎉 Transformer benchmark completed successfully!")
print("="*80)
